In [29]:
# !python -m pip install --upgrade pip
# !pip install --upgrade pip setuptools wheel
# !pip install transformers tiktoken blobfile llama-stack
# !pip install google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2
# !pip install torch fairscale fire sentencepiece protobuf pyyaml

*get LLAMA_REPO_URL from https://www.llama.com/llama-downloads/*

- $env:PATH += ";%USERPROFILE%\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\Scripts"
- $env:LLAMA_REPO_URL = "https://llama3-2-lightweight.llamameta.net/*?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoieDAwZXVhc3h2MmN6cm1jZXZ2aGhkYmN5IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvbGxhbWEzLTItbGlnaHR3ZWlnaHQubGxhbWFtZXRhLm5ldFwvKiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1NTUxMzgyNX19fV19&Signature=n9xX1DRBt-hhv5oe2nqi0ldRmFPAMEV3g0UFUvufLXRed28Jj-38f%7EUV%7EnThjxGGKklDLkOt7VxS-9Kh02nYjk0wMWldD0tKmnxj8XBATG13Vt4s%7E41WfpUEMv1rJtvb81KqwgVUdetoNwOD1dUYfQPhUrvV2L8TvKqwyPat0Paz0lnSma1evhhSEltkJD15e6VQcHpf6Gbq454lqjx2NbtVxj1Wv0hjl4V5U2kC1S7nt8QSFLtZ6rB-9nNUM-yEZKWYym12PNwXrTbP6omRTr4nCHX22xVgZFGCm8aNtZ5V6IxETWiDhKdf3srReGefbH6sXan8BY5oEivNn9J4yw__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1486291695705908"
- llama model list
- llama model download --source meta --model-id Llama3.2-3B-Instruct --meta-url $env:LLAMA_REPO_URL
- llama model verify-download --model-id Llama3.2-3B-Instruct

- cd D:\Projects\my-jupyter-notebook\gpt-test
- mv "$env:USERPROFILE\.llama\checkpoints\Llama3.2-3B-Instruct" "llama-models/"

In [ ]:
# !python llama-models/convert_llama_weights_to_hf.py --input_dir llama-models/Llama3.2-3B-Instruct --model_size 3B --llama_version 3.2 --output_dir llama-models/Llama3.2-3B-Instruct-transformed

Converting the tokenizer.
Saving a LlamaTokenizerFast to llama-models/Llama3.2-3B-Instruct-transformed.
Converting the model.
Fetching all parameters from the checkpoint at llama-models/Llama3.2-3B-Instruct.
Loading the checkpoint in a Llama model.
Saving in the Transformers format.
Saving to disk.



Loading checkpoint shards: 100%|##########| 29/29 [00:00<00:00, 92.67it/s]


In [ ]:
from transformers import LlamaForCausalLM, AutoTokenizer
import torch
import time

import os
os.chdir("D:/Projects/my-jupyter-notebook/gpt-test")

model_name = "Llama3.2-3B-Instruct"
transformed_model_path = f"llama-models/{model_name}-transformed"
model = LlamaForCausalLM.from_pretrained(transformed_model_path)
tokenizer = AutoTokenizer.from_pretrained(transformed_model_path)

def chat(model, device, prompt, max_new_tokens, silent=False):
    print(f"Prompt: {prompt}") if not silent else None
    model = model.to(device)
    start_time = time.time()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generate_ids = model.generate(inputs.input_ids, max_new_tokens=max_new_tokens, eos_token_id=None)
    response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    elapsed_time = time.time() - start_time
    print(f"Response ({elapsed_time:.4f}s on {model.device}) : {response[0]}\n{'-'*20}\n") if not silent else None
    return response[0]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
prompt = "Tell me an interesting fact about elephants"
response = chat(model, torch.device("cuda"), prompt, 100)
response = chat(model, torch.device("cpu"), prompt, 100)

Prompt: Tell me an interesting fact about elephants
Response (4.1522s on cuda:0) : Tell me an interesting fact about elephants
I'd love to learn something new about these magnificent creatures. Here's a fascinating fact:

**Elephants have a highly developed brain structure, and their brain-to-body mass ratio is similar to that of humans.** In fact, studies have shown that the cerebral cortex of an elephant is similar in structure and organization to that of a human brain, with areas dedicated to processing sensory information, memory, and social behavior. This suggests that elephants may possess a level
--------------------

Prompt: Tell me an interesting fact about elephants
Response (37.2174s on cpu) : Tell me an interesting fact about elephants
Elephants have a highly developed sense of empathy and compassion. They have been known to show signs of grief and mourning when they lose a family member, and they will often go to great lengths to care for and comfort each other in times of

In [ ]:
prompt = "Tell me an interesting fact about elephants"
response = chat(model, torch.device("cuda"), prompt, 500)

Prompt: Tell me an interesting fact about elephants
Response (22.3962s on cuda:0) : Tell me an interesting fact about elephants
Here's one: Did you know that elephants have a highly developed brain and are considered one of the smartest animals on Earth? In fact, their brain-to-body mass ratio is similar to that of humans, which is one of the highest among all land animals. This means that elephants have a large amount of brain power relative to their body size, allowing them to exhibit complex behaviors, problem-solving abilities, and even empathy and self-awareness. In fact, studies have shown that elephants are able to recognize themselves in mirrors, a cognitive ability that is rare in the animal kingdom. Isn't that cool? 
Here's another interesting fact about elephants: 
Elephants have a highly developed sense of smell, with a trunk that contains millions of olfactory receptors, which is more sensitive than a bloodhound's nose. This allows them to detect subtle changes in their en

In [54]:
prompt = "Tell me an interesting fact about elephants"
response = chat(model, torch.device("cuda"), prompt, 3000)

Prompt: Tell me an interesting fact about elephants
Response (258.4570s on cuda:0) : Tell me an interesting fact about elephants
I'd be happy to! Here's an interesting fact about elephants:

**Elephants have a highly developed brain and can recognize themselves in a mirror!**

In fact, studies have shown that elephants are one of the few animals, along with chimpanzees and dolphins, that possess a level of self-awareness, known as mirror self-recognition. This means that they can look at themselves in a mirror and recognize their own reflection. This cognitive ability is often considered a hallmark of human intelligence, and it's fascinating to think that elephants, with their massive size and complex social behavior, possess this level of self-awareness.

Would you like to know more about elephants or is there something else you'd like to explore?
--------------------



# Text to Speech

In [9]:
# !pip install pyttsx3

In [2]:
import pyttsx3

def say(response, rate):
    engine = pyttsx3.init()
    engine.setProperty("rate", rate)
    engine.setProperty("volume", 1.0)
    engine.say(response)
    engine.runAndWait()
    pyttsx3.engine.Engine.stop(engine)

In [ ]:
prompt = "Tell me an interesting fact about elephants"
say(chat(model, torch.device("cuda"), prompt, max_new_tokens=100), rate=150)

Prompt: Tell me an interesting fact about elephants
Response (4.3249s on cuda:0) : Tell me an interesting fact about elephants
Here's one: Elephants have a highly developed sense of empathy and can recognize and respond to the emotions of other elephants. They have been known to show compassion and kindness to each other, often going out of their way to comfort a distressed or grieving elephant. In fact, studies have shown that elephants are more empathetic towards each other than humans are towards each other. Isn't that fascinating? 
What do you think about this fact? Do you find
--------------------



# Speech to Text

In [6]:
# !pip install SpeechRecognition pyaudio

In [3]:
import speech_recognition as sr

def listen(phrase_time_limit):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Say something...")
        audio = recognizer.listen(source, phrase_time_limit=phrase_time_limit)
    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        print("Sorry, could not understand audio")
    except sr.RequestError:
        print("API unavailable or no internet")
    return None

In [ ]:
say(chat(model, torch.device("cuda"), listen(), max_new_tokens=100), rate=150)

Say something...
Prompt: hello how r u
Response (4.3159s on cuda:0) : hello how r u today
I'm doing great, thanks for asking! Just got back from a nice walk and enjoying the sunshine. How about you? How's your day going so far?

(By the way, I noticed you used the informal "r u" instead of "are you". That's totally fine in informal conversations, but just a heads up for those who might not be used to it!) 

So, what's on your mind today? Want to chat about something in particular
--------------------



# Chatbot loop

In [ ]:
while True:
    prompt = listen()
    if not prompt:
        say("No input detected, please try again.", rate=150)
        continue
    if prompt.lower().strip() in ["exit", "quit", "goodbye", "bye", "good bye"]:
        say("Good bye!", rate=150)
        print("Exiting chatbot loop.")
        break
    response = chat(model, torch.device("cuda"), prompt, max_new_tokens=30)
    say(response, rate=150)

Say something...

Prompt: hello how r u
Response (1.2610s on cuda:0) : hello how r u today?
I'm doing alright, thanks for asking! Just got back from a walk outside and enjoying the sunshine. How about
--------------------

Say something...

Prompt: I am fine
Response (1.4429s on cuda:0) : I am fine, but my friend is experiencing a bit of a crisis. I want to be supportive, but I don't know how to help
--------------------

Say something...

Prompt: who is he
Response (1.4085s on cuda:0) : who is he
I think you meant to ask "Who is he?" but didn't specify who "he" is. Could you please provide
--------------------

Say something...

Prompt: I am in your friend
Response (1.2259s on cuda:0) : I am in your friend's house, and you are the host, and you've invited me over for a casual dinner party. The evening is
--------------------

Say something...

Prompt: how is the weather today
Response (1.2677s on cuda:0) : how is the weather today in different parts of the world
The weather can vary g

# Chatbot loop with context

In [11]:
conversations = ""
while True:
    prompt = listen(phrase_time_limit=10)
    if not prompt:
        say("No input detected, please try again.", rate=150)
        continue
    print(f"User said: {prompt}")
    if prompt.lower().strip() in ["exit", "quit", "goodbye", "bye", "good bye", "cancel"]:
        say("Good bye!", rate=150)
        print("Exiting chatbot loop.")
        break
    conversations += f"{prompt}"
    response = chat(model, torch.device("cuda"), conversations, max_new_tokens=20, silent=True)
    response = response[len(conversations):].strip()
    print(f"Bot said: {response}")
    conversations += f"\n{response}."
    say(response, rate=150)

Say something...
User said: how are you
Bot said: today?
I'm doing well, thanks for asking. It's a beautiful day outside, and I
Say something...
User said: and you
Bot said: ? 

P.S. I hope you're not too busy to chat. I'd love to hear
Say something...
User said: what do you want to
Bot said: talk about?

## Step 1: Identify the greeting and response format
The greeting is a friendly
Say something...
User said: goodbye
Exiting chatbot loop.
